<a href="https://colab.research.google.com/github/mirjanaa/Bee-Colony-Optimization/blob/main/money_dynamic_programing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
import random
import math
from matplotlib import pyplot as plt

In [23]:
def dynamic_programing_algorithm(coins, target):
  N = len(coins)
  M = target

  dp = [[0 for j in range(M+1)] for i in range(N+1)]

  for target in range(1, M+1):
    dp[0][target] = float('inf')
  for n in range(1, N+1):
    dp[n][0] = 0
    for i in range(1, M+1):
      dp[n][i] = dp[n-1][i]
      if coins[n-1] <= i and dp[n-1][i-coins[n-1]] != float('inf'):
        dp[n][i] = min(dp[n][i], 1 + dp[n-1][i-coins[n-1]])

  indicator = False
  for i in range(M, 0, -1):
    if dp[N][i] != float('inf'):
      indicator = True
      return dp[N][i], i
  
  if not indicator:
    print
    return float('inf'), 0

In [24]:
class FoodSource:
  def __init__(self, bounds, obj_function, target, beta_parameter):
    self.din_1 = random.randrange(bounds[0]+1)
    self.din_2 = random.randrange(bounds[1]+1)
    self.din_5 = random.randrange(bounds[2]+1)
    self.din_10 = random.randrange(bounds[3]+1)
    self.din_20 = random.randrange(bounds[4]+1)

    self.position = np.array([self.din_1, self.din_2, self.din_5, self.din_10, self.din_20])
    self.target = target
    self.obj_value = obj_function(self.position, self.target, beta_parameter)
    self.fitness = self.calculate_fitness()
    self.trial = 0
    self.probability = 0

  def __lt__(self, other):
    return self.obj_value < other.obj_value

  def __gt__(self, other):
    return self.fitness > other.fitness

  def calculate_fitness(self):
    if self.obj_value >= 0:
      return 1 / (1 + self.obj_value)
    return 1 + abs(self.obj_value)

In [25]:
def obj_function(position, target, beta):
  alpha = 1 #parametar tezine vrednosti blizine resenja
  #beta = 1  #parametar tezine broja novcica

  din_1 = position[0]
  din_2 = position[1]
  din_5 = position[2]
  din_10 = position[3]
  din_20 = position[4]

  value = 1*din_1 + 2*din_2 + 5*din_5 + 10*din_10 + 20*din_20
  count = din_1 + din_2 + din_5 + din_10 + din_20

  return alpha*abs(target - value) + beta*(count)

In [26]:
def generate_new_solution(i, population, bounds, option, beta_parameter):
  j = random.randrange(len(bounds))
  partner = random.randrange(len(population))

  while i == partner:
    partner = random.randrange(len(population))

  old_fitness = population[i].fitness
  old_objective_value = population[i].obj_value

  old_j = population[i].position[j]
  phi = random.randrange(0, 2) #random
  if option == 'average':
    new_j = math.floor((population[i].position[j] + population[partner].position[j]) / 2) + phi
  elif option == 'max':
    new_j = max(population[i].position[j], population[partner].position[j]) + phi

  population[i].position[j] = new_j
  population[i].position[j] = np.clip(population[i].position[j], 0, bounds[j])

  new_objective_value = obj_function(population[i].position, population[i].target, beta_parameter)

  population[i].obj_value = new_objective_value
  population[i].fitness = population[i].calculate_fitness()

  if population[i].fitness > old_fitness:
    population[i].trial = 0
  else:
    population[i].position[j] = old_j
    population[i].obj_value = old_objective_value
    population[i].fitness = old_fitness
    population[i].trial += 1

In [27]:
def ABC_algorithm(population_size, num_of_iters, limit, obj_function, option, target, beta_parameter):

  POPULATION_SIZE = population_size
  NUM_OF_ITERATIONS = num_of_iters
  LIMIT = limit

  population = [FoodSource(bounds, obj_function, target, beta_parameter) for _ in range(POPULATION_SIZE)]

  best_solution = min(population)

  obj_values = []
  fitness_values = []

  for k in range(NUM_OF_ITERATIONS):
    #### Employed Bee Phase ####
    for i in range(POPULATION_SIZE):
      generate_new_solution(i, population, bounds, option, beta_parameter)

    #### Onlooker Bee Phase ####
    max_fitness = max([population[i].fitness for i in range(POPULATION_SIZE)])
    for i in range(POPULATION_SIZE):
      population[i].probability = 0.9 * (population[i].fitness / max_fitness) + 0.1

    m = 0
    n = 0

    while m < POPULATION_SIZE and n < POPULATION_SIZE:
      rand_value = random.uniform(0, 1)
      if rand_value < population[n].probability:
        generate_new_solution(n, population, bounds, option, beta_parameter)
        m += 1
      n = (n % POPULATION_SIZE) + 1

    best_solution = min(min(population), best_solution)

    #### Scout Bee Phase ####
    max_index = -1
    max_trial = float('-inf')
    for index, p in enumerate(population):
      if p.trial > max_trial:
        max_trial = p.trial
        max_index = index


    if population[max_index].trial > LIMIT:
      population[max_index] = FoodSource(bounds, obj_function, target, beta_parameter)

    best_solution = min(min(population), best_solution)

    obj_values.append(best_solution.obj_value)
    fitness_values.append(best_solution.fitness)

  print("position: ", [i for i in best_solution.position])
  print("objective value: ", best_solution.obj_value)
  print("fitness value: ", best_solution.fitness)


  return best_solution, obj_values, fitness_values

In [28]:
bounds = [0, 51000, 0, 0, 0]
best_solution, _, _ = ABC_algorithm(500, 1000, 5, obj_function, 'max', 100000, 1)

position:  [0, 50000, 0, 0, 0]
objective value:  50000
fitness value:  1.999960000799984e-05


In [ ]:
bounds = [0, 51000, 0, 0, 0]
values = [1, 2, 5, 10, 20] 
target = 100000
coins = []

for i in range(len(bounds)):
  for j in range(bounds[i]):
    coins.append(values[i])

dynamic_programing_algorithm(coins, target)